In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()
print(session)

In [ ]:
use role SYSADMIN;

In [ ]:
db_name = 'ACCELERATORS'
strmlt_role = '_STRMLT_RL'

In [ ]:
session.sql(f"create database if not exists {db_name}").collect()
session.sql(f"grant usage on database {db_name} to role {strmlt_role}").collect()

session.sql(f"create schema if not exists {db_name}.STRMLT").collect()
session.sql(f"grant usage on schema {db_name}.STRMLT to role {strmlt_role}").collect()
session.sql(f"grant create streamlit on schema {db_name}.STRMLT to role {strmlt_role}").collect()

session.sql(f"create schema if not exists {db_name}.STP").collect()
session.sql(f"grant usage on schema {db_name}.STP to role {strmlt_role}").collect()
session.sql(f"grant all on schema {db_name}.STP to role {strmlt_role}").collect()

In [ ]:
session.sql("CREATE OR REPLACE WAREHOUSE _STRMLT_WH WAREHOUSE_SIZE=XSMALL AUTO_RESUME=TRUE AUTO_SUSPEND = 120 INITIALLY_SUSPENDED=TRUE ;").collect()
session.sql(f"grant usage on WAREHOUSE _STRMLT_WH to role {strmlt_role}").collect()

In [ ]:
use role USERADMIN;

In [ ]:
try:
    session.sql(f"""
        create role _STRMLT_RL;
    """).collect()
    session.sql(f"""
        grant role STRMLT_RL to role USERADMIN;
    """)
except Exception as e:
    print(e)

In [ ]:
use role ACCOUNTADMIN;

In [ ]:
session.sql(f"grant database role SNOWFLAKE.SECURITY_VIEWER to role _STRMLT_RL;")

In [ ]:
for db in session.sql("select DATABASE_NAME from SNOWFLAKE.INFORMATION_SCHEMA.DATABASES where DATABASE_OWNER = 'SYSADMIN'").collect():
    print(db['DATABASE_NAME'])
    try:
        ### ROLE CAN ACCESS DATABASE
        session.sql(f"""
            grant usage on database {db['DATABASE_NAME']} to role _STRMLT_RL;
        """).collect()
        ### ROLE CAN ACCESS ALL SCHEMAS IN DATBASE
        session.sql(f"""
            grant usage on all schemas in database {db['DATABASE_NAME']} to role _STRMLT_RL;
        """).collect()
        ### ROLE CAN CREATE SECRETS
        session.sql(f"""
            grant create secret on all schemas in database {db['DATABASE_NAME']} to role _STRMLT_RL;
        """).collect()
        ### ROLE CAN UPDATE SECRETS
        session.sql(f"""
            grant ownership on all secrets in database {db['DATABASE_NAME']} to role _STRMLT_RL;
        """)
        ### ROLE CAN CREATE SECRETS
        session.sql(f"""
            grant create network rule on all schemas in database {db['DATABASE_NAME']} to role _STRMLT_RL;
        """).collect()
    except Exception as e:
        print(e)

In [ ]:
use role ACCOUNTADMIN;

In [ ]:
session.sql("grant CREATE EXTERNAL ACCESS INTEGRATION on account to role _STRMLT_RL").collect()